In [17]:
import os
key = os.urandom(16)
for i in range (0,len(key)):
    print(key[i])

192
223
26
213
189
80
137
55
242
108
255
198
10
218
155
184


In [18]:
sub_Box = [#OriginalNib
          [0b0000, 0b0001, 0b0010, 0b0011, 0b0100, 0b0101, 0b0110, 0b0111], 
          #Subistituation
          [0b1001, 0b0100, 0b1010, 0b1011, 0b1101, 0b0001, 0b1000, 0b0101],
          #OriginalNib
          [0b1000, 0b1001, 0b1010, 0b1011, 0b1100, 0b1101, 0b1110, 0b1111], 
          #Subistituation
          [0b0110, 0b0010, 0b0000, 0b0011, 0b1100, 0b1110, 0b1111, 0b0111]]

In [19]:
for i in range (0, len(sub_Box)):
        for y in range(0, len(sub_Box[i])):
            sub_Box[i][y] = format(sub_Box[i][y], "04b")

In [20]:
#word4 for = word0 xor subNib(rotNib(word3))
def rotNib(key):
    temp1 = key[0]
    temp2 = key[1]
    
    key[0] = key[2]
    key[1] = key[3]
    
    key[2] = temp1
    key[3] = temp2
    
    return key

In [21]:
def SubNib(key):    
    for i in range(0, len(key)):
        Div1 = key[i][:4]
        Div2 = key[i][4:]
        
        flagDiv1 = 0
        flagDiv2 = 0
        
        y = 0
        while y < len(sub_Box):
            for z in range(0, len(sub_Box[y])):
                if flagDiv1 == 0:
                    if key[i][:4] == sub_Box[y][z]:
                        Div1 = sub_Box[y + 1][z]
                        flagDiv1 = 1
                if flagDiv2 == 0:
                    if key[i][4:] == sub_Box[y][z]:
                        Div2 = sub_Box[y + 1][z]
                        flagDiv2 = 1
                
                if flagDiv1 == 1 & flagDiv2 == 1:
                    key[i] = Div1 + Div2
                    break
            y = y + 2
    return key

In [22]:
def decimalToBinary(n):
    # converting decimal to binary
    # and removing the prefix(0b)
    return '{:032b}'.format(n)

In [23]:
def ExpandKey(key, sub_Box):
    KeyArray = []
    for i in range (0,len(key)):
        KeyArray.append(format(key[i], "08b"))
    
    #for i in range (0,len(KeyArray)):
     #   print(KeyArray[i])
        
        
    MultiArrayKey = []
    rows,cols = (4,4)
    i = 0;
    for r in range(rows):
        col = []
        for c in range(cols):
            col.append(KeyArray[i])
            i+=1
        MultiArrayKey.append(col)

    #print(MultiArrayKey)
    FW = 0
    
    for i in range (0, 9):
        
        print('last word: ', MultiArrayKey[len(MultiArrayKey ) - 1])
        Rotated_Word = rotNib(MultiArrayKey[len(MultiArrayKey ) - 1])
        print('rotated word: ', Rotated_Word)
        
        subistutedWord = SubNib(Rotated_Word)
        
        print('subistuted Word', subistutedWord)
        
        FirstWordInt = int(MultiArrayKey[FW][0] + MultiArrayKey[FW][1] +  MultiArrayKey[FW][2] + MultiArrayKey[FW][3], 2)
        SecondWordInt = int(subistutedWord[0] + subistutedWord[1] + subistutedWord[2] + subistutedWord[3], 2)
        
        print('First Word Int: ', FirstWordInt)
        print('Second Word Int: ', SecondWordInt)
        
        new_Word = FirstWordInt ^ SecondWordInt
    
        new_Word = decimalToBinary(new_Word)
        
        print(new_Word)
        
        col = []
        
        col.append(new_Word[:8])
        col.append(new_Word[8:16])
        col.append(new_Word[16:24])
        col.append(new_Word[24:])
        
        MultiArrayKey.append(col)
        
        for z in range (0, 3):
            
            FW += 1
            num = len(MultiArrayKey) - 1
            
            FirstWordInt = int(MultiArrayKey[FW][0] + MultiArrayKey[FW][1] +  MultiArrayKey[FW][2] + MultiArrayKey[FW][3], 2)
            SecondWordInt = int(MultiArrayKey[num][0] + MultiArrayKey[num][1] +  MultiArrayKey[num][2] + MultiArrayKey[num][3], 2)
            
            new_Word = FirstWordInt ^ SecondWordInt
            
            new_Word = decimalToBinary(new_Word)
            
            print(new_Word)
            
            col = []
            
            col.append(new_Word[:8])
            col.append(new_Word[8:16])
            col.append(new_Word[16:24])
            col.append(new_Word[24:])
            
            MultiArrayKey.append(col)
        
        
    for i in range (0, len(MultiArrayKey)):
        print('Word number', i ,':', MultiArrayKey[i])
    return MultiArrayKey

In [24]:
def text2Unicode(text):
    w, h = 16, 8
    l = [0 for x in range(16)] 

    for i in range(16):
        
        
        l1 = ' '.join(map(bin,bytearray(text[i],'utf8')))
        l[i] = l1[2:]
        #l = list(l1[2:])
        #for i1 in range(len(l1)):
        #    lst2[i][i1]=l[i1]
        #    print(l[i1])
    w, h = 16, 8
    lst2 = [[0 for x in range(8)] for y in range(16)] 

    for row in range(16):
        r1 = list(l[row])
        for i in range(0, len(r1)):
            lst2[row][i] = int(r1[i]) 

    return lst2

In [25]:
def mixColumns(a, b, c, d,ii,l3):
    l3[ii]    = (gmul(a, 0x02) ^ gmul(b, 0x03) ^ gmul(c, 0x01) ^ gmul(d, 0x01))
    l3[ii+4]  = (gmul(a, 0x01) ^ gmul(b, 0x02) ^ gmul(c, 0x03) ^ gmul(d, 0x01))
    l3[ii+8]  = (gmul(a, 0x01) ^ gmul(b, 0x01) ^ gmul(c, 0x02) ^ gmul(d, 0x03))
    l3[ii+12] = (gmul(a, 0x03) ^ gmul(b, 0x01) ^ gmul(c, 0x01) ^ gmul(d, 0x02))
    l3[ii]   ='{:08b}'.format(l3[ii])
    l3[ii+4] ='{:08b}'.format(l3[ii+4])
    l3[ii+8] ='{:08b}'.format(l3[ii+8])
    l3[ii+12]='{:08b}'.format(l3[ii+12])
    return l3

def gmul(a, b):
    if b == 1:
        return a
    binary = '{:08b}'.format(a)
    
    tmp = (a << 1) & 0xff
    print('temp After', binary)
    
    if b == 2:
        if a < 128:
            #binary = '{:08b}'.format(a)
            #print('a < 128:', binary)
            return tmp
        else:
            #binary = '{:0b}'.format(a)
            #print('a > 128:', binary)
            return tmp ^ 0x1b
        
#        return tmp if a < 128 else tmp & 0xff ^ 0x1b
    if b == 3:
        return gmul(a, 2) ^ a

def printHex(val):
    return print('{:02x}'.format(val), end=' ')
#    return print('{:02x}'.format(val), end=' ')
def MIX_COLUMNS1(mat1):
    L1 = [0 for x in range(16)]   
    for i1 in range(0,16):
        firstnumber = ""
        for i in range(0,8):
            firstnumber += str(mat1[i1][i])

        L1[i1] =  firstnumber            
    print(L1)
    hexadecimal_string = [0 for x in range(16)]
    for i1 in range(0,16):
        decimal_representation = int(L1[i1], 2)
        hexadecimal_string[i1] = hex(decimal_representation)
        hexadecimal_string[i1] = int(hexadecimal_string[i1],16)

    print(hexadecimal_string)
    ii = 0
    l3 = [0 for x in range(16)]

    for i1 in range(0,4):
        l2 = mixColumns(hexadecimal_string[ii], hexadecimal_string[ii+4], 
                   hexadecimal_string[ii+8], hexadecimal_string[ii+12],ii,l3)
        ii +=1
    print(l2)
    ii = 0
    w, h = 8, 16
    lst2 = [[0 for x in range(w)] for y in range(h)]
    res = [[0 for x in range(w)] for y in range(h)]
    for row in range(16):
        r1 = list(l2[row])
        for i in range(0, len(r1)):
            lst2[row][i] = int(r1[i]) 
    return lst2 
#l3 = [0 for x in range(16)]
#for i1 in range(0,16):
#    l3 = '{:08b}'.format(l2[i1])
#print(l3)

In [26]:
def XOR(Key1,plaintext):
    w= 4
    w, h = 16, 8
    lst2 = [[0 for x in range(8)] for y in range(16)] 
    w, h = 16, 8
    lst3 = [[0 for x in range(8)] for y in range(16)] 
    L2 = [0 for x in range(w)]   
    r1 = [0 for x in range(8)]   
    r2 = [0 for x in range(8)]   
    for i in range(0,16):
        io = 0
        for i1 in range(0,8):
            L3 = " "
            binOne = plaintext[i][i1]
            binTwo = Key1[i][i1]
            binOne1 = str(binOne)
            binTwo1 = str(binTwo)
            L33 = int(binOne1,2) ^ int(binTwo1,2)
            L3 = bin(L33)
            r = str(L3[2:])
            lst3[i][i1] = int(r)
            #    print("\nResult = ",lst3)
    return lst3

In [27]:
def Sub_Nib(Roundkey,sboxIN,sboxOUT):
    w, h = 16, 8
    lst2 = [[0 for x in range(w)] for y in range(h)] 
    i2 = 0
    for i1 in range(0, 16):
        if (Roundkey[i2][0] == sboxIN[i1][0] and Roundkey[i2][1] == sboxIN[i1][1] and Roundkey[i2][2] == sboxIN[i1][2] and Roundkey[i2][3] == sboxIN[i1][3]):
            for i in range(0, 4):
                 Roundkey[i2][i] = sboxOUT[i1][i]
                    
            i2 += 1
    i3 = 0
    for i1 in range(0, 16):
        if (Roundkey[i3][4] == sboxIN[i1][0] and Roundkey[i3][5] == sboxIN[i1][1] and Roundkey[i3][6] == sboxIN[i1][2] and Roundkey[i3][7] == sboxIN[i1][3]):
            i4 = 0
            for i in range(4, 8):
                Roundkey[i3][i] = sboxOUT[i1][i4]
                i4 += 1

            i3 += 1
    return Roundkey

In [28]:
import numpy as np

def Shift_Row(shift):
    w, h = 16, 8
    lst2 = [[0 for x in range(w)] for y in range(h)] 
    for i in range(0, 8):
        lst2[0][i] = shift[4][i]
        shift[4][i] = shift[5][i]
        shift[5][i] = shift[6][i]
        shift[6][i] = shift[7][i]
        shift[7][i] = lst2[0][i]
        ##2nd row
        lst2[1][i] =  shift[8][i]
        shift[8][i] = shift[10][i]
        shift[10][i] = lst2[1][i]
        ##2nd row net column
        lst2[2][i] =  shift[9][i]
        shift[9][i] = shift[11][i]
        shift[11][i] = lst2[2][i]
        ##4th row
        lst2[2][i] = shift[12][i]
        shift[12][i] = shift[15][i]
        lst2[3][i] = shift[13][i]
        shift[13][i] = lst2[2][i]
        lst2[4][i] = shift[14][i]
        shift[14][i] = lst2[3][i]
        lst2[5][i] = shift[15][i]
        shift[15][i] = lst2[4][i]
    return shift

In [29]:
sboxIN = [[0,0,0,0],#1 
          [0,0,0,1],#2 
          [0,0,1,0],#3 
          [0,0,1,1],#4 
          [0,1,0,0],#5 
          [0,1,0,1],#6 
          [0,1,1,0],#7 
          [0,1,1,1],#8 
          [1,0,0,0],#9 
          [1,0,0,1],#10
          [1,0,1,0],#11
          [1,0,1,1],#12
          [1,1,0,0],#13
          [1,1,0,1],#14
          [1,1,1,0],#15
          [1,1,1,1]]#16

sboxOUT = [ [1,0,0,1],#1 
            [0,1,0,0],#2 
            [1,0,1,0],#3 
            [1,0,1,1],#4 
            [1,1,0,1],#5 
            [0,0,0,1],#6 
            [1,0,0,0],#7 
            [0,1,0,1],#8 
            [0,1,1,0],#9 
            [0,0,1,0],#10
            [0,0,0,0],#11
            [0,0,1,1],#12
            [1,1,0,0],#13
            [1,1,1,0],#14
            [1,1,1,1],#15
            [0,1,1,1]]#16
MultiArrayKey = ExpandKey(key, sub_Box)

plain_text = input("Enter a string to be encoded : ")
length = len(plain_text)
plain_text_split = []
for i in range(length//16):
    plain_text_split.append(plain_text[0+16*i:16+16*i])
if not length%16==0:        
    plain_text_split.append(plain_text[16*(length//16):])
if len(plain_text_split[-1])<16:
    while(len(plain_text_split[-1])<16):
        plain_text_split[-1]+=' '
for sub_string in plain_text_split : 
    A0 = text2Unicode(sub_string)
    print(A0)

last word:  ['00001010', '11011010', '10011011', '10111000']
rotated word:  ['10011011', '10111000', '00001010', '11011010']
subistuted Word ['00100011', '00110110', '10010000', '11100000']
First Word Int:  3235846869
Second Word Int:  590778592
11100011111010011000101000110101
01011110101110010000001100000010
10101100110101011111110011000100
10001111111000110110110000100100
last word:  ['10001111', '11100011', '01101100', '00100100']
rotated word:  ['01101100', '00100100', '10001111', '11100011']
subistuted Word ['10001100', '10101101', '01100111', '11111011']
First Word Int:  590778592
Second Word Int:  2360174587
10101111100110111111011100011011
01001100011100100111110100101110
00010010110010110111111000101100
10111110000111101000001011101000
last word:  ['10111110', '00011110', '10000010', '11101000']
rotated word:  ['10000010', '11101000', '10111110', '00011110']
subistuted Word ['01101010', '11110110', '00111111', '01001111']
First Word Int:  2899705028
Second Word Int:  17945229

In [30]:
def encrypt(A0,MultiArrayKey):
    w, h = 16, 8
    lst2 = [[0 for x in range(8)] for y in range(16)] 
    Key2 = [0 for y in range(16)] 
    row1 = 0
    row = 0
    for row2 in range (9):
        Key2[row1] = MultiArrayKey[row][0]
        Key2[row1+1] = MultiArrayKey[row][1]
        Key2[row1+2] = MultiArrayKey[row][2]
        Key2[row1+3] = MultiArrayKey[row][3]
        Key2[row1+4] = MultiArrayKey[row+1][0]
        Key2[row1+5] = MultiArrayKey[row+1][1]
        Key2[row1+6] = MultiArrayKey[row+1][2]
        Key2[row1+7] = MultiArrayKey[row+1][3]
        Key2[row1+8] = MultiArrayKey[row + 2][0]
        Key2[row1+9] = MultiArrayKey[row + 2][1]
        Key2[row1+10] = MultiArrayKey[row + 2][2]
        Key2[row1+11] = MultiArrayKey[row + 2][3]
        Key2[row1+12] = MultiArrayKey[row + 3][0]
        Key2[row1+13] = MultiArrayKey[row + 3][1]
        Key2[row1+14] = MultiArrayKey[row + 3][2]
        Key2[row1+15] = MultiArrayKey[row + 3][3]

        for row in range(16):
                r1 = list(Key2[row])
                for i in range(0, len(r1)):
                    lst2[row][i] = int(r1[i]) 
        print(A0)

        plaintext = A0
        Key1 = lst2
        #sub-bytes
        shift = Sub_Nib(plaintext,sboxIN,sboxOUT)
        print("Sub_Nib",shift)
        #shift-rows
        mix=Shift_Row(shift)
        print("Shift_Row",mix)
        #mix-columns
        v1 = lol = MIX_COLUMNS1(mix)
        print("mix_column",v1)
        #round key
        Roundkey = XOR(Key1,v1)
        print("Roundkey",Roundkey)
        plaintext = Roundkey
        text = ""
        for i1 in range(16):
            for i in range(8):
                text+=chr(int(plaintext[i1][i])) # chr converts unicode integer to unicode character
        row += 4

    return Roundkey

In [31]:
cipher_text = encrypt(A0,MultiArrayKey)
print(cipher_text)

[[1, 1, 0, 0, 0, 0, 1, 0], [1, 1, 1, 1, 0, 0, 1, 0], [1, 1, 0, 1, 1, 1, 1, 0], [1, 1, 1, 0, 1, 0, 1, 0], [1, 1, 0, 0, 0, 1, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0]]
Sub_Nib [[1, 1, 0, 0, 1, 0, 1, 0], [0, 1, 1, 1, 0, 0, 1, 0], [1, 1, 0, 1, 1, 1, 1, 0], [1, 1, 1, 0, 1, 0, 1, 0], [1, 1, 0, 0, 0, 1, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0]]
Shift_Row [[1, 1, 0, 0, 1, 0, 1, 0], [0, 1, 1, 1, 0, 0, 1, 0], [1, 1, 0, 1, 1, 1, 1, 0], [1, 1, 1, 0, 1, 0, 1, 0], [1, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0

In [32]:
def BinaryToDecimal(binary):
    string = int(binary, 2)     
    return string
# initializing a empty string for
# storing the string data
str_data =' '
temp_data =' '
# slicing the input and converting it
# in decimal and then converting it in string
for i in range(0, 16):
    temp_data =' ' 
    # slicing the bin_data from index range [0, 6]
    # and storing it in temp_data
#    for i1 in range(0,16):
    for i1 in range(0,8):
        temp_data += str(cipher_text[i][i1])
    # passing temp_data in BinarytoDecimal() function
    # to get decimal value of corresponding temp_data
    decimal_data = BinaryToDecimal(temp_data)
      
    # Decoding the decimal value returned by
    # BinarytoDecimal() function, using chr()
    # function which return the string corresponding
    # character for given ASCII value, and store it
    # in str_data
    str_data = str_data + chr(decimal_data)
 
# printing the result
print("The Binary value after string conversion is:",
       str_data)

The Binary value after string conversion is:  j 0¦r
×8®kIèSÔ
